In [ ]:
import urllib.request,sys,time
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json

In [ ]:
url = 'https://sbr.com.sg/stocks'
req = Request(url , headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()
time.sleep(2)
soup = BeautifulSoup(webpage, "html.parser")
noOfPages = soup.find('a',attrs={'title':'Go to last page'})['href'][6:]
print(noOfPages)

In [ ]:
def scrapeURL(url):
    req = Request(url , headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    time.sleep(2)
    soup = BeautifulSoup(webpage, "html.parser")
    metadata = soup.find('article')
    output_dict = dict()
    
    #Title
    title = metadata.find('h1',attrs={'class': 'nf__title'}).text.strip()
    output_dict['title'] = title
    
    #Text
    text = metadata.find('div',attrs={'class': 'nf__description'}).text.strip()
    output_dict['text'] = text
    
    #Date
    script = soup.find('script',attrs={'type': 'application/ld+json'}).text
    dictionary = json.loads(script)
    date = dictionary['@graph'][0]['datePublished']
    output_dict['date'] = date
    
    return output_dict

In [ ]:
articles_df = pd.DataFrame(columns = ['Title', 'Text', 'Link', 'Date'])
for page in range(0,int(noOfPages)):
    print('Processing page :', page)
    url = 'https://sbr.com.sg/stocks?page='+str(page)
    print(url)
    
    req = Request(url , headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    
    time.sleep(2)
    
    soup = BeautifulSoup(webpage, "html.parser")
    soup = soup.find('main',attrs={'class':'main-content'})
    links=soup.find_all('div',attrs={'class':'item with-border-bottom'})
   
    for j in links:
        link = j.find("h2",attrs={'class':'item__title size-24'}).find('a')['href'].strip()
        output_dict = scrapeURL(link)
        
        #Other metadata
#         article.summary
#         article.keywords 
#         article.top_image
#         article.images 
        
        articles_df = articles_df.append({'Link' : link, 'Title': output_dict['title'], 'Text': output_dict['text'], 'Date': output_dict['date']}, ignore_index = True)


In [ ]:
articles_df.to_csv('sbr_articles_stocks', index=0)